In [1]:
import shioaji as sj
from pymongo import MongoClient
from datetime import datetime, timedelta
from pandas import DataFrame
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
def getFollowedAssets(followedAssets=None):
    try:
        if followedAssets is None:
#             self.logger.info('GetAssets')
            table = getTable(setting, schema,'StockList')
            updateDate = sorted(table.distinct('UpdateDate'))[-1]
            datas = list(table.find({'Industry':{'$ne':''}}))
            return [x['Ticker'] for x in datas if x['UpdateDate'] == updateDate]
            # with open(os.path.join(parent, 'FollowedAssets.txt'), 'r') as f:
            #     lines = f.readlines()[-1]
            #     return lines.split(',')[1:]
    except Exception as e:
        print(e)
        pass
#         self.sendError('getFollowedAssets')
    else:
        return followedAssets

In [3]:
def getTable(setting, schema, table_name):
    try:
        user = setting['user']
        pwd = setting['pwd']
        ip = setting['ip']
        port = setting['port']
        client = MongoClient(f'mongodb://{user}:{pwd}@{ip}:{port}')
        schema = client['admin'][schema]
        table = schema[table_name]
    except:
        self.sendError('getTable')
    return table
setting = {
    'user':'kevin83321',
    'pwd':'j7629864',
    'ip':'192.168.2.173',
    'port':'49153'
}
schema = 'TWSE'
# if table_name is None:
table_name = 'historicalPrice'
table = getTable(setting, schema, table_name)

In [4]:
end_date = datetime.today()
entry_threshold=.06
v_threshold = 1000
_followedAssets = []
open_thresholds = {}
# tickers = self.getFollowedAssets(followedAssets)

# end_date = datetime(2021,4,27) #.today()
tickers = getFollowedAssets()
end_date -= timedelta(1)
start_date = end_date + timedelta(-60)
table = getTable(setting, schema, table_name)
datas = list(table.find({'Ticker':{'$in':tickers}, 'Date':{'$gte':start_date.strftime("%Y-%m-%d"), '$lte':end_date.strftime("%Y-%m-%d")}}))
# datas = readStockDataInter(tickers,start_date, end_date)
Master_df = DataFrame(datas)
del Master_df['_id']

df_map = {}
for ticker in tickers:
    df_map[ticker] = Master_df[Master_df.Ticker==ticker]

possible_strong = {}

for k, df in df_map.items():
    if df.empty: continue
    if df.shape[0] < 20: continue
    # print(df)
    df['Close'] = df.Close.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
    df['High'] = df.High.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
    df['Open'] = df.Open.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
    df['Low'] = df.Low.apply(lambda x: float('nan') if '-' in str(x) else x).fillna(method='ffill').astype(float)
    df['Volume'] = df.Volume.apply(lambda x: 0 if '-' in str(x) else x).fillna(method='ffill').astype(float)
    df['MA5'] = df.Close.rolling(5).mean()
    df['MA10'] = df.Close.rolling(10).mean()
    df['MA20'] = df.Close.rolling(20).mean()
    df['DI'] = (df.High+df.Low+2*df.Close) / 4
    df['Amp'] = (df.High-df.Low) / df.Low
    df['AmpStd'] = df.Amp.rolling(5).std()
    df['DiStd'] = df['DI'].rolling(10).std()
    df['VMA5'] = df.Volume.rolling(5).mean()
    # df['MA5Slope'] = df['MA5'].pct_change()
    # df['MA10Slope'] = df['MA10'].pct_change()
    # df['MA20Slope'] = df['MA20'].pct_change()
    MAStd = np.std([df.MA5.iloc[-1]-df.MA10.iloc[-1],df.MA5.iloc[-1]-df.MA20.iloc[-1], df.MA10.iloc[-1]-df.MA20.iloc[-1]])
    open_threshold = max(df['Close'].iloc[-5], df['Close'].iloc[-10], df['Close'].iloc[-20])
    if (open_threshold / df['Close'].iloc[-1]) < (1 + entry_threshold) and (df.VMA5.iloc[-1]) / 1000 >= v_threshold:
#             if df['MA5Slope'].iloc[-1] >= 0.01:# and df['MA10Slope'].iloc[-1] >= 0 and df['MA20Slope'].iloc[-1] >= 0:
        temp_ma5 = np.mean(df['Close'].iloc[-4:].tolist() + [open_threshold])
        possible_strong[k] = (open_threshold, MAStd, df.DiStd.iloc[-1]/df['Close'].iloc[-1], df.AmpStd.iloc[-1])
        _followedAssets.append(k)
        open_thresholds[k] = open_threshold

In [5]:
len(open_thresholds.keys())

526

In [5]:
api1 = sj.Shioaji()

In [6]:
api1.login("F128497445", "j7629864")

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


[FutureAccount(person_id='F128497445', broker_id='F002000', account_id='1473661', signed=True, username='鄭圳宏'),
 Account(account_type=<AccountType.H: 'H'>, person_id='F128497445', broker_id='9A92', account_id='0011645', username='鄭圳宏'),
 StockAccount(person_id='F128497445', broker_id='9A92', account_id='0231901', signed=True, username='鄭圳宏\u3000\u3000')]

In [7]:
from shioaji import TickSTKv1, Exchange

@api1.on_tick_stk_v1()
def quote_callback1(exchange: Exchange, tick:TickSTKv1):
    print(f"from api 1, Exchange: {exchange}, Tick: {tick}")

api1.quote.set_on_tick_stk_v1_callback(quote_callback1)

In [8]:
for i in range(500,526):
    ticker = sorted(open_thresholds.keys())[i]
    api1.quote.subscribe(
        api1.Contracts.Stocks[ticker], 
        quote_type = sj.constant.QuoteType.Tick, # or 'tick'
        version = sj.constant.QuoteVersion.v1 # or 'v1'
    )
    api1.quote.subscribe(
        api1.Contracts.Stocks[ticker], 
        quote_type = sj.constant.QuoteType.BidAsk, # or 'tick'
        version = sj.constant.QuoteVersion.v1 # or 'v1'
    )

Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/8112 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/8112 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/OTC/8121 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/OTC/8121 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/8150 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/8150 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/TSE/8163 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/TSE/8163 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: TIC/v1/STK/*/OTC/8171 | Event: Subscribe or Unsubscribe ok
Response Code: 200 | Event Code: 16 | Info: QUO/v1/STK/*/OTC/8171 | Event